Краулер для nplus1

In [1]:
# Импортируем необходимые библиотеки:
import requests # http-запросы,
import re # регулярные выражения,
from bs4 import BeautifulSoup # удаление тегов html,
from tqdm import tqdm # красотуля для анализа прогресса.
import time

In [3]:
# Функия принимает адрес статьи на nplus1.ru и возвращает текст статьи и метаинформацию по ней.
def getArticleTextNPlus1(adr):
    r = requests.get(adr) # Получаем текст статьи, ...
#print(r.text)
    tables=re.split("</div>",re.split('="tables"', r.text)[1])[0] # ... берем блок с метаинформацией, ...
# ... извлекаем из блока метаинформацию. 
    t1=re.split("</time>", re.split("<time", tables)[1])[0]
    n_time=re.split("</span>", re.split("<span>", t1)[1])[0]
    n_date=re.split("</span>", re.split("<span>", t1)[2])[0]
    n_rubr=re.split(">", re.split("</a>", re.split("<a href", tables)[1])[0])[1]
    n_diff=re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
# Достаем заголовок, автора и текст статьи.
    n_head=re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    n_author=re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    n_text=re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

# "Откусываем" оставшиеся теги.
    beaux_text=BeautifulSoup(n_text, "lxml")
    n_text=beaux_text.get_text() 

# print(n_time, n_date, n_rubr, n_diff)
# print(n_head)
# print(n_author)
# print(n_text)
    return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
# def getArticleTextNPlus1(adr):


In [4]:

# Получаем текст страницы, которая содержит ссылки на все статьи этого дня (в примере - 11.01.2016).
r = requests.get('https://nplus1.ru/news/2016/01/11/')
# Каждая ссылка на статью оормлена с помощью тега <article>
refs=re.split('<article class="item item-news item-news', r.text)
database=[]
# Если ссылки вообще есть, ...
if len(refs)>0:
    total=0
# ..., то перебираем их все, кроме первой части - там только мусор. 
    for t in refs[1:]:
        href=re.split('"', t)[6]
        #print(href)
# Извлекаем текст статьи, кладем в список, считаем общую длину полученных новостей.
        database.append(getArticleTextNPlus1("https://nplus1.ru"+href))
        total+=len(database[-1][6])
        print(database[0])
        time.sleep(2)
        print(total)
# Вот и всё.

['19:08', '11 Янв. 2016', 'Космос', '3.7', '', 'Астрономы построили первую диаграмму роста Млечного Пути', 'Группа астрономов во главе\nс Мелиссой Несс из Института Макса Планка в Гедельберге, Германия, построили\nпервую в истории диаграмму роста Галактики Млечный Путь. Результаты работы были\nпредставлены на встрече Американского астрономического общества, прошедшей в\nКиссимми, штат Флорида, США, а сообщение об исследовании опубликовано на сайте SDSS.org.\n\nДиаграмма\nроста помогает определить, как развивалась наша галактика с момента своего\nпоявления до современной спиральной структуры. Для этого астрономам пришлось\nисследовать область длиной в 50 тысяч световых лет, что составляет почти\nполовину диаметра галактического диска, и определить возраст более 70 тысяч\nзвезд.\nУченые\nобнаружили, что ближе к центру находятся старые звезды, которые сформировались,\nкогда Галактика была молодой и небольшой по размеру. По мере удаления от\nгалактического ядра встречаются все более юные з